In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import pandas as pd
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Create a dataset with 5000 rows
n_rows = 5000

# Define the columns
columns = [
    'timestamp', 'market', 'historical_volume', 'projected_growth',
    'external_factors', 'idle_resources_eu', 'idle_resources_asia',
    'allocated_resources_eu', 'allocated_resources_asia', 'predicted_volume_increase'
]

# Generate data
data = {
    'timestamp': pd.date_range(start='1/1/2023', periods=n_rows, freq='H'),
    'market': np.random.choice(['US', 'EU', 'Asia'], size=n_rows),
    'historical_volume': np.random.randint(100, 10000, size=n_rows),
    'projected_growth': np.random.uniform(0.01, 0.5, size=n_rows),  # Growth rate between 1% and 50%
    'external_factors': np.random.uniform(0.8, 1.2, size=n_rows),   # External factor scaling between 0.8 and 1.2
    'idle_resources_eu': np.random.randint(10, 100, size=n_rows),   # Idle resources in EU
    'idle_resources_asia': np.random.randint(10, 100, size=n_rows), # Idle resources in Asia
    'allocated_resources_eu': np.zeros(n_rows),  # To be filled based on logic
    'allocated_resources_asia': np.zeros(n_rows),# To be filled based on logic
    'predicted_volume_increase': np.zeros(n_rows) # To be filled based on logic
}

# Create DataFrame
df = pd.DataFrame(data)

# Fill in the allocated resources and predicted volume increase
for i in range(n_rows):
    if df.loc[i, 'market'] == 'Asia':
        df.loc[i, 'predicted_volume_increase'] = df.loc[i, 'historical_volume'] * df.loc[i, 'projected_growth'] * df.loc[i, 'external_factors']
        df.loc[i, 'allocated_resources_asia'] = df.loc[i, 'idle_resources_asia'] * 0.25
        df.loc[i, 'allocated_resources_eu'] = df.loc[i, 'idle_resources_eu'] * 0.25
    else:
        df.loc[i, 'predicted_volume_increase'] = df.loc[i, 'historical_volume'] * df.loc[i, 'projected_growth']
        df.loc[i, 'allocated_resources_asia'] = df.loc[i, 'idle_resources_asia'] * 0.1
        df.loc[i, 'allocated_resources_eu'] = df.loc[i, 'idle_resources_eu'] * 0.1

# Display the first few rows of the dataset
df.head()


,timestamp,market,historical_volume,projected_growth,external_factors,idle_resources_eu,idle_resources_asia,allocated_resources_eu,allocated_resources_asia,predicted_volume_increase
0,2023-01-01 00:00:00,Asia,5430,0.478263,1.046518,96,89,24.00,22.25,2717.775464
1,2023-01-01 01:00:00,US,7971,0.388324,0.862064,57,14,5.70,1.40,3095.329541
2,2023-01-01 02:00:00,Asia,8335,0.422762,1.030669,91,24,22.75,6.00,3631.791031
3,2023-01-01 03:00:00,Asia,3712,0.382697,1.130508,73,30,18.25,7.50,1605.967690
4,2023-01-01 04:00:00,US,2718,0.092423,0.817010,25,67,2.50,6.70,251.206897


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, GRU
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Preparing the data for LSTM

# Feature selection
features = ['historical_volume', 'projected_growth', 'external_factors']
target = 'predicted_volume_increase'

# Scaling and reshaping data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[features])
target_data = df[target].values.reshape(-1, 1)

# Creating sequences for LSTM
sequence_length = 10  # Number of time steps

X = []
y = []

for i in range(sequence_length, len(scaled_data)):
    X.append(scaled_data[i-sequence_length:i])
    y.append(target_data[i])

X = np.array(X)
y = np.array(y)

model = Sequential()
model.add(GRU(units=100, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(GRU(units=100))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(1))

model.compile(optimizer=RMSprop(learning_rate=0.001), loss='mean_squared_error')

early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=3, min_lr=1e-6)

model.fit(X, y, epochs=50, batch_size=32, callbacks=[early_stopping, reduce_lr])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Epoch 1/50
156/156 [==============================] - 9s 24ms/step - loss: 2887708.0000 - lr: 0.0010
Epoch 2/50
156/156 [==============================] - 3s 19ms/step - loss: 2874616.5000 - lr: 0.0010
Epoch 3/50
156/156 [==============================] - 3s 19ms/step - loss: 2850780.0000 - lr: 0.0010
Epoch 4/50
156/156 [==============================] - 3s 20ms/step - loss: 2814743.2500 - lr: 0.0010
Epoch 5/50
156/156 [==============================] - 4s 26ms/step - loss: 2767768.2500 - lr: 0.0010
Epoch 6/50
156/156 [==============================] - 3s 19ms/step - loss: 2709773.7500 - lr: 0.0010
Epoch 7/50
156/156 [==============================] - 3s 19ms/step - loss: 2640942.0000 - lr: 0.0010
Epoch 8/50
156/156 [==============================] - 3s 19ms/step - loss: 2562387.5000 - lr: 0.0010
Epoch 9/50
156/156 [==============================] - 4s 28ms/step - loss: 2477236.5000 - lr: 0.0010
Epoch 10/50
156/156 [==============================] - 3s 19ms/step - loss: 2381686.7500 - 

In [ ]:
!pip install gym stable-baselines3 shimmy gymnasium

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import gym
import gymnasium as gym  # Use gymnasium
import numpy as np
from stable_baselines3 import DQN

class ResourceAllocationEnv(gym.Env):
    def __init__(self, df):
        super(ResourceAllocationEnv, self).__init__()
        self.df = df
        self.current_step = 0
        self.action_space = gym.spaces.Discrete(10)
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(3,), dtype=np.float32)

    def reset(self, **kwargs):
        self.current_step = 0
        return self._next_observation(), {}

    def _next_observation(self):
        obs = self.df.iloc[self.current_step][['idle_resources_eu', 'idle_resources_asia', 'predicted_volume_increase']].values
        return obs

    def step(self, action):
        self.current_step += 1
        done = self.current_step >= len(self.df)
        truncated = False  # Set to True if you have specific conditions to truncate

        obs = self._next_observation() if not done else np.zeros(self.observation_space.shape)
        reward = self._get_reward(action)

        return obs, reward, done, truncated, {}

    def _get_reward(self, action):
        reward = 1 - np.abs(self.df.iloc[self.current_step]['allocated_resources_asia'] - action)
        return reward

env = ResourceAllocationEnv(df)

model = DQN('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=1000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Prepare DataFrame
data = {
    'timestamp': pd.date_range(start='1/1/2023', periods=5000, freq='H'),
    'market': np.random.choice(['US', 'EU', 'Asia'], size=5000),
    'historical_volume': np.random.randint(100, 10000, size=5000),
    'projected_growth': np.random.uniform(0.01, 0.5, size=5000),
    'external_factors': np.random.uniform(0.8, 1.2, size=5000),
    'idle_resources_eu': np.random.randint(10, 100, size=5000),
    'idle_resources_asia': np.random.randint(10, 100, size=5000),
}

df = pd.DataFrame(data)

# Add calculated columns
df['predicted_volume_increase'] = df.apply(
    lambda row: row['historical_volume'] * row['projected_growth'] * row['external_factors']
    if row['market'] == 'Asia' else row['historical_volume'] * row['projected_growth'], axis=1
)

# Create features
df['day_of_week'] = df['timestamp'].dt.dayofweek
df['market_status'] = df['timestamp'].dt.hour.isin([9, 16]).astype(int)

# Features and target
features = ['day_of_week', 'market_status', 'historical_volume', 'projected_growth', 'external_factors', 'idle_resources_eu']
target = 'predicted_volume_increase'

X = df[features]
y = df[target]  # Use actual predicted volume increase

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training the Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)  # Use RandomForestRegressor for continuous output
model.fit(X_train, y_train)

# Predicting and evaluating the model
y_pred = model.predict(X_test)

# Combine features and predictions into a DataFrame for evaluation
results_df = X_test.copy()
results_df['predicted_volume_increase'] = y_pred

results_df.head()

,day_of_week,market_status,historical_volume,projected_growth,external_factors,idle_resources_eu,predicted_volume_increase
1501,5,0,1001,0.110694,1.171243,85,128.319050
2586,1,0,3666,0.393087,0.897411,94,1444.278462
2653,4,0,1691,0.389779,0.899485,79,679.064370
1055,0,0,3456,0.460128,1.184426,93,1569.586857
705,0,1,4853,0.134822,1.056868,98,662.400719


In [ ]:
# Sample input data for all days of the week
sample_data_all_days = [
    {
        'day_of_week': 0,  # Monday
        'market_status': 1,  # open
        'historical_volume': 3000,
        'projected_growth': 0.3,
        'external_factors': 1.05,
        'idle_resources_eu': 20
    },
    {
        'day_of_week': 1,  # Tuesday
        'market_status': 1,  # open
        'historical_volume': 4000,
        'projected_growth': 0.25,
        'external_factors': 1.1,
        'idle_resources_eu': 40
    },
    {
        'day_of_week': 2,  # Wednesday
        'market_status': 1,  # open
        'historical_volume': 5000,
        'projected_growth': 0.2,
        'external_factors': 1.15,
        'idle_resources_eu': 30
    },
    {
        'day_of_week': 3,  # Thursday
        'market_status': 1,  # open
        'historical_volume': 6000,
        'projected_growth': 0.15,
        'external_factors': 1.2,
        'idle_resources_eu': 50
    },
    {
        'day_of_week': 4,  # Friday
        'market_status': 1,  # open
        'historical_volume': 7000,
        'projected_growth': 0.1,
        'external_factors': 1.05,
        'idle_resources_eu': 20
    },
    {
        'day_of_week': 5,  # Saturday
        'market_status': 0,  # close
        'historical_volume': 8000,
        'projected_growth': 0.4,
        'external_factors': 0.9,
        'idle_resources_eu': 60
    },
    {
        'day_of_week': 6,  # Sunday
        'market_status': 0,  # close
        'historical_volume': 9000,
        'projected_growth': 0.35,
        'external_factors': 1.15,
        'idle_resources_eu': 50
    }
]

# Create a DataFrame for the samples
sample_df_all_days = pd.DataFrame(sample_data_all_days)

# Predict using the trained Random Forest model
predicted_increases = model.predict(sample_df_all_days)

# Combine features and target into a single DataFrame
sample_df_all_days['predicted_volume_increase'] = predicted_increases
sample_df_all_days

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,day_of_week,market_status,historical_volume,projected_growth,external_factors,idle_resources_eu,predicted_volume_increase
0,0,1,3000,0.30,1.05,20,935.653924
1,1,1,4000,0.25,1.10,40,1039.592480
2,2,1,5000,0.20,1.15,30,1057.398422
3,3,1,6000,0.15,1.20,50,935.265102
4,4,1,7000,0.10,1.05,20,706.613226
5,5,0,8000,0.40,0.90,60,3203.939398
6,6,0,9000,0.35,1.15,50,3409.693832


In [ ]:
# Convert day_of_week to string representation
day_names = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday',
              5: 'Saturday', 6: 'Sunday'}
sample_df_all_days['day_name'] = sample_df_all_days['day_of_week'].map(day_names)

# Filter for weekdays and find the top 2 predicted_volume_increase
top_weekdays = sample_df_all_days[sample_df_all_days['day_of_week'].isin(range(0, 5))] \
                                  .nlargest(2, 'predicted_volume_increase')

print("Top 2 predicted volume increases for weekdays:")
top_weekdays[['day_name', 'predicted_volume_increase', 'idle_resources_eu']]


Top 2 predicted volume increases for weekdays:


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,day_name,predicted_volume_increase,idle_resources_eu
2,Wednesday,1057.398422,30
1,Tuesday,1039.592480,40


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Print messages for the top 2 predictions
for index, row in top_weekdays.iterrows():
    projected_growth = row['projected_growth'] * 100
    day_name = row['day_name']
    predicted_increase = row['predicted_volume_increase']
    idle_resources = row['idle_resources_eu']

    print(f"A {projected_growth:.1f}% growth in trade volume is expected in the ASIA region on {day_name} and hence, "
          f"{idle_resources} units of idle resources is requested from EU region.")

A 20.0% growth in trade volume is expected in the ASIA region on Wednesday and hence, 30 units of idle resources is requested from EU region.
A 25.0% growth in trade volume is expected in the ASIA region on Tuesday and hence, 40 units of idle resources is requested from EU region.
